# Building a Spotify Recommendation Engine for Music Labels
## Recommendation System and Evaluation
### Flatiron School Data Science Program Phase 4 Project<br>Justin Williams & Khyatee Desai

In [49]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from zipfile import ZipFile 
from sklearn import set_config
set_config(print_changed_only=False, display=None)
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
% matplotlib inline
import seaborn as sns
sns.set_style('darkgrid')
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import os
import sys

### Get Data From Spotify

In [61]:
os.environ["SPOTIPY_CLIENT_ID"] = "a8561b2ef375442f8f89527b255f4c78"
os.environ["SPOTIPY_CLIENT_SECRET"] = "566a59fd72764b9580f8767c5bc0ca7f"

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

spotify = spotipy.Spotify(auth_manager=SpotifyClientCredentials())

name = 'rangerover porches'

results = spotify.search(q=name, type='track')
# print(results)

In [116]:
# in the works

### Import Cleaned Datasets

In [117]:
# import main songs dataset
main_df = pd.read_csv('data/cleaned_data_main.csv',index_col=[0])

# drop unnecesary features
main_df.drop(['key_mode', 'decade', 'modes', 'letter_keys',  'year', 
         'release_date', 'mode', 'loudness', 'key', 'id', 'explicit', 
         'tempo', 'duration_ms', 'speechiness', 'popularity' ], axis=1, inplace=True)

In [118]:
# import record labels dataset
labels_df = pd.read_csv('data/cleaned_data.csv',index_col=[0])

# drop unnecesary features
labels_df.drop(['key_mode', 'decade', 'modes', 'letter_keys',  'year', 
         'release_date', 'mode', 'loudness', 'key', 'id', 'explicit', 
         'tempo', 'duration_ms', 'speechiness', 'popularity' ], axis=1, inplace=True)

### Make sample user playlist

In [103]:
# using 5 example songs from the main dataset
playlist_df = main_df[(main_df.name.isin(['Night Beat','Re: Stacks','Stillness Is The Move', 'Slip Away',
                'Love Is Mystical'])) & (main_df.artists.isin(['Bon Iver', 'Chuck Berry',
                    'Cold War Kids','Dirty Projectors','Perfume Genius']))]
playlist_df

,acousticness,artists,danceability,energy,instrumentalness,liveness,name,valence,scaled_speech,scaled_duration,scaled_loudness,scaled_tempo,scaled_pop,A Major,A Minor,Ab Major,Ab Minor,B Major,B Minor,Bb Major,Bb Minor,C Major,C Minor,D Major,D Minor,Db Major,Db Minor,E Major,E Minor,Eb Major,Eb Minor,F Major,F Minor,F# Major,F# Minor,G Major,G Minor,1920s,1930s,1940s,1950s,1960s,1970s,1980s,1990s,2000s,2010s,2020s
18570,0.91300,Chuck Berry,0.507,0.3920,0.941000,0.1130,Night Beat,0.703,0.296318,0.029359,0.764827,0.892507,0.23,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
31910,0.37100,Perfume Genius,0.603,0.5090,0.000027,0.0627,Slip Away,0.572,0.127871,0.029680,0.832762,0.559595,0.61,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
55374,0.00462,Cold War Kids,0.567,0.8630,0.000048,0.2070,Love Is Mystical,0.722,0.151642,0.038762,0.858852,0.434387,0.60,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0
62169,0.10900,Dirty Projectors,0.569,0.5420,0.000004,0.2040,Stillness Is The Move,0.530,0.549192,0.057279,0.849049,0.312625,0.41,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
96542,0.97900,Bon Iver,0.676,0.0812,0.416000,0.1180,Re: Stacks,0.314,0.163118,0.073412,0.581850,0.512321,0.64,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


### Make Dead Oceans Dataframe

In [105]:
# dataframe of all Dead Oceans songs
ocean_df = labels_df[labels_df.label == 'Dead Oceans']
ocean_df.head()

,acousticness,danceability,energy,instrumentalness,liveness,name,valence,scaled_speech,scaled_duration,scaled_loudness,scaled_tempo,scaled_pop,A Major,A Minor,Ab Major,Ab Minor,B Major,B Minor,Bb Major,Bb Minor,C Major,C Minor,D Major,D Minor,Db Major,Db Minor,E Major,E Minor,Eb Major,Eb Minor,F Major,F Minor,F# Major,F# Minor,G Major,G Minor,1920s,1930s,1940s,1950s,1960s,1970s,1980s,1990s,2000s,2010s,2020s,artist,label
3919,0.162000,0.181,0.486,0.2030,0.1110,Machine Gun,0.0545,0.152872,0.048459,0.716686,0.633260,0.40,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,Slowdive,Dead Oceans
3920,0.098700,0.172,0.745,0.0073,0.2350,40 Days,0.4910,0.153282,0.035013,0.774223,0.757644,0.37,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,Slowdive,Dead Oceans
3921,0.195000,0.129,0.647,0.2920,0.1040,Machine Gun,0.0393,0.156970,0.048824,0.792687,0.633165,0.41,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,Slowdive,Dead Oceans
3922,0.001370,0.431,0.482,0.4410,0.2380,Ballad Of Sister Sue,0.1230,0.126232,0.049432,0.816804,0.524579,0.38,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,Slowdive,Dead Oceans
3923,0.000057,0.227,0.480,0.8350,0.0477,Catch The Breeze,0.1880,0.159020,0.047115,0.841203,0.379780,0.37,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,Slowdive,Dead Oceans


### Calculate Cosine Similarities between user playlist and Dead Oceans songs

In [106]:
# turn both dataframes into numpy arrays
playlist_array = np.array(playlist_df.drop(['name', 'artists'], axis=1))
label_array = np.array(ocean_df.drop(['artist','name','label'], axis=1))

In [107]:
# turn both arrays into sparse matrices
playlist_sparse = sparse.csr_matrix(playlist_array)
label_sparse = sparse.csr_matrix(label_array)

# get cosine similarities between user playlist and record label
cosim = cosine_similarity(playlist_sparse, label_sparse,dense_output=True)
print(cosim[0])

[0.41725962 0.44058502 0.43945933 0.44629123 0.46998448 0.42616374
 0.52013517 0.43897967 0.46968424 0.57943389 0.52753198 0.51112629
 0.501619   0.48604798 0.57567217 0.54186608 0.54485576 0.47879578
 0.46176429 0.5234061  0.46243494 0.58832096 0.43790728 0.51030357
 0.38095126 0.62952489 0.37217698 0.39019201 0.44538295 0.64848205
 0.41835028 0.59267187 0.4442569  0.71788021 0.45640818 0.48222531
 0.47887033 0.42578593 0.70592829 0.44086193 0.57484672 0.57559412
 0.4944472  0.47711631 0.54565493 0.59813884 0.54863266 0.57668559
 0.52198825 0.54298583 0.59757559 0.5284611  0.45390556 0.5270894
 0.55502917 0.5328001  0.47620957 0.47950022 0.48059881 0.66846413
 0.5287695  0.46204234 0.45508129 0.76327526 0.49434258 0.51870572
 0.54187491 0.47272966 0.41602542 0.40006322 0.70354566 0.38142567
 0.56339016 0.32553798 0.61705959 0.41187208 0.62261361 0.51431157
 0.49466633 0.51317297 0.50243734]


### Sort by Highest Similarities and map output to song names/artists

In [108]:
for i in range(len(cosim)):
    print('Dead Oceans Recs based on', playlist_df.iloc[i]['name'], 'by',playlist_df.iloc[i]['artists'],':\n' )
    print(sorted(list(zip(cosim[i],ocean_df['name'], ocean_df['artist'])), reverse=True)[:3])
    print('================================================================================================================')

Dead Oceans Recs based on Night Beat by Chuck Berry :

[(0.7632752574993004, 'Hey, Snow White', 'Destroyer'), (0.7178802103919614, 'June On The West Coast', 'Bright Eyes'), (0.7059282877813741, 'At The Bottom Of Everything', 'Bright Eyes')]
Dead Oceans Recs based on Slip Away by Perfume Genius :

[(0.9814496281995175, 'Motion Sickness', 'Phoebe Bridgers'), (0.9458096552062896, 'Townie', 'Mitski'), (0.7664558828907302, 'Beautiful Strangers', 'Kevin Morby')]
Dead Oceans Recs based on Love Is Mystical by Cold War Kids :

[(0.7773262089988421, 'Texas Sun', 'Khruangbin'), (0.7754942829144683, 'Kaputt', 'Destroyer'), (0.7743949637623367, 'Sugaree', 'Phosphorescent')]
Dead Oceans Recs based on Stillness Is The Move by Dirty Projectors :

[(1.0000000000000002, 'Stillness Is The Move', 'Dirty Projectors'), (0.9495021228951672, 'Something Vague', 'Bright Eyes'), (0.9487395751466929, 'At The Bottom Of Everything', 'Bright Eyes')]
Dead Oceans Recs based on Re: Stacks by Bon Iver :

[(0.97264825369

# Put it all in a function

In [112]:
def recommend(playlist, record_label):
    '''input 1: playlist formatted as a list of tuples [(track_name, artist), (track_name, artist),etc]
        input 2: record label name (string)'''
    
    # make dataframe using user's playlist
    playlist_df = main_df[(main_df['name'].isin(list(zip(*playlist))[0])) & (main_df['artists'].isin(list(zip(*playlist))[1]))]
    
    # make dataframe of all songs in record label
    label_df = labels_df[labels_df.label == record_label]
    
    # turn both dataframes into numpy arrays
    playlist_array = np.array(playlist_df.drop(['name', 'artists'], axis=1))
    label_array = np.array(labels_df.drop(['artist','name','label'], axis=1))
    
    # turn both arrays into sparse matrices
    playlist_sparse = sparse.csr_matrix(playlist_array)
    label_sparse = sparse.csr_matrix(label_array)

    # get cosine similarities between user playlist and record label
    cosim = cosine_similarity(playlist_sparse, label_sparse,dense_output=True)
    
    # print results
    for i in range(len(cosim)):
        print(record_label,'tracks similar to', playlist_df.iloc[i]['name'], 'by',playlist_df.iloc[i]['artists']+':\n' )
        print(sorted(list(zip(cosim[i],labels_df['name'], labels_df['artist'])), reverse=True)[:3])
        print('================================================================================================================')

### Test with sample playlist

In [113]:
sample_playlist = [('Night Beat', 'Bon Iver'),
                     ('Re: Stacks', 'Chuck Berry'),
                     ('Stillness Is The Move', 'Dirty Projectors'),
                     ('Love Is Mystical', 'Cold War Kids'),
                     ('Slip Away', 'Perfume Genius')]

recommend(sample_playlist, 'Sub Pop')

Sub Pop tracks similar to Night Beat by Chuck Berry:

[(0.9999999999999999, 'Night Beat', 'Chuck Berry'), (0.9640043020267528, 'What Now, What Next, Where To', 'Elvis Presley'), (0.9602153318170781, 'Father Ramirez - Padre Ramirez', 'Ennio Morricone')]
Sub Pop tracks similar to Slip Away by Perfume Genius:

[(1.0000000000000002, 'Slip Away', 'Perfume Genius'), (0.9884259860310165, 'Head On (Hold On To Your Heart)', 'Man Man'), (0.9841178868107674, 'Might Not', 'Belly')]
Sub Pop tracks similar to Love Is Mystical by Cold War Kids:

[(0.9999999999999998, 'Love Is Mystical', 'Cold War Kids'), (0.9897162329445989, 'Dead and Gone', 'The Black Keys'), (0.9716727070575977, 'Cold Little Heart - Radio Edit', 'Michael Kiwanuka')]
Sub Pop tracks similar to Stillness Is The Move by Dirty Projectors:

[(1.0000000000000002, 'Stillness Is The Move', 'Dirty Projectors'), (0.9907392751806603, 'Personal Space Invader', 'AJJ'), (0.9902575577169858, 'Little Acorns', 'The White Stripes')]
Sub Pop tracks si

### Test with Khyatee's playlist

In [115]:
# didn't work because none of these songs are in main df :-)
sample_playlist = [('Polly', 'Moses Sumney'),
                     ('Goodbye', 'Porches'),
                     ('rangerover', 'Porches'),
                     ('Tal Uno', 'Barrie'),
                     ("Angel's Song", 'Arlo Parks'),
                     ('Randy', 'Big Thief'),
                      ('Butterfly', 'Adrianne Lenker'),
                      ('All of Me Wants All of You', 'Sufjan Stevens'),
                      ('Cola', 'Arlo Parks')]

recommend(sample_playlist, 'Sub Pop')

ValueError: Found array with 0 sample(s) (shape=(0, 46)) while a minimum of 1 is required by check_pairwise_arrays.

### Test with Justin's playlist

# Different Similarity Metrics?

# Evaluation - RMSE?